In [13]:
const dataLoop = (
  data: Data,
  {
    level = 0,
    parentNode = null,
    hook = () => {},
  }: {
    level: number;
    parentNode: null | Node;
    hook: ({
      data,
      parentNode,
      level,
    }: {
      data: Data;
      parentNode: null | Node;
      level: number;
    }) => void;
  }
) => {
  hook({
    data,
    parentNode,
    level,
  });

  data?.nodes?.forEach((node) => {
    if (node.children) {
      dataLoop(node.children as Data, {
        level: level + 1,
        parentNode: node,
        hook,
      });
    }
  });
};

const getLinkId = (link: { source: string; target: string }) =>
  `${link.source}-${link.target}`;

const DEFAULT_RADIUS = 4;

function calculateContainerRadius({
  childRadius = DEFAULT_RADIUS,
  childLen = 1,
  padding = 8,
}) {
  if (childLen <= 1) {
    return childRadius + padding;
  }

  // 计算最小立方体边长，该立方体可以容纳所有子球体
  const minCubeSize = Math.ceil(Math.cbrt(childLen));

  // 计算立方体的半边长（考虑球体间距）
  const halfCubeSize = (minCubeSize - 1) * childRadius;

  // 计算从立方体中心到角落的距离
  const cornerDistance = Math.sqrt(3) * halfCubeSize;

  // 容器球体半径 = 角落距离 + 子球体半径 + 内部填充
  const containerRadius = cornerDistance + childRadius + padding;

  return containerRadius * 1.6;
}


In [14]:
const prepareData = (data: Data) => {
  const nodeMap: Record<
      string,
      { node: Node; parent: string | null; level: number }
    > = {},
    linkMap: Record<
      string,
      { link: Link; parent: string | null; level: number }
    > = {};

  let levelMaxChildLen: number[] = [];

  dataLoop(data, {
    level: 0,
    parentNode: null,
    hook: ({ data, parentNode, level }) => {
      const nodes = data?.nodes || [];
      const links = data?.links || [];
      levelMaxChildLen[level] = Math.max(
        levelMaxChildLen[level] || 0,
        nodes.length || 0
      );
      nodes.forEach((node) => {
        const id = node.id;
        nodeMap[id] = { node, parent: parentNode?.id || null, level };
      });
      links.forEach((link) => {
        const id = getLinkId(link);
        linkMap[id] = { link, parent: parentNode?.id || null, level };
      });
    },
  });

  levelMaxChildLen = levelMaxChildLen.filter((len) => !!len);

  const levelRadius = [DEFAULT_RADIUS];

  for (let index = levelMaxChildLen.length - 1; index > 0; index--) {
    const levelChildLen = levelMaxChildLen[index];
    const radius = calculateContainerRadius({
      childLen: levelChildLen,
      childRadius: levelRadius[0],
    });
    levelRadius.unshift(radius);
  }

  return {
    nodeMap,
    linkMap,
    levelMaxChildLen,
    levelRadius,
  };
};


In [15]:
const graphData = {
  nodes: [
    {
      id: "0",
      children: {
        nodes: [
          { id: "00", parent: "0" },
          { id: "01", parent: "0" },
          { id: "02", parent: "0" },
        ],
        links: [
          { target: "01", source: "00" },
          { target: "02", source: "00" },
        ],
      },
    },
    {
      id: "1",
      children: {
        nodes: [
          { id: "10", parent: "1" },
          { id: "11", parent: "1" },
          { id: "12", parent: "1" },
          { id: "13", parent: "1" },
        ],
        links: [
          { target: "11", source: "10" },
          { target: "12", source: "10" },
          { target: "13", source: "11" },
        ],
      },
    },
    {
      id: "2",
      children: {
        nodes: [
          { id: "20", parent: "2" },
          { id: "21", parent: "2" },
          { id: "22", parent: "2" },
          { id: "23", parent: "2" },
          { id: "24", parent: "2" },
        ],
        links: [
          { target: "21", source: "20" },
          { target: "22", source: "21" },
          { target: "23", source: "20" },
          { target: "24", source: "21" },
        ],
      },
    },
    {
      id: "3",
      children: {
        nodes: [
          { id: "30", parent: "3" },
          { id: "31", parent: "3" },
          { id: "32", parent: "3" },
        ],
        links: [
          { target: "31", source: "30" },
          { target: "32", source: "30" },
        ],
      },
    },
  ],
  links: [
    { target: "1", source: "0" },
    { target: "2", source: "0" },
    { target: "3", source: "0" },
  ],
};


In [16]:
console.log(prepareData(graphData));


{
  nodeMap: {
    "0": {
      node: { id: "0", children: { nodes: [Array], links: [Array] } },
      parent: null,
      level: 0
    },
    "1": {
      node: { id: "1", children: { nodes: [Array], links: [Array] } },
      parent: null,
      level: 0
    },
    "2": {
      node: { id: "2", children: { nodes: [Array], links: [Array] } },
      parent: null,
      level: 0
    },
    "3": {
      node: { id: "3", children: { nodes: [Array], links: [Array] } },
      parent: null,
      level: 0
    },
    "10": { node: { id: "10", parent: "1" }, parent: "1", level: 1 },
    "11": { node: { id: "11", parent: "1" }, parent: "1", level: 1 },
    "12": { node: { id: "12", parent: "1" }, parent: "1", level: 1 },
    "13": { node: { id: "13", parent: "1" }, parent: "1", level: 1 },
    "20": { node: { id: "20", parent: "2" }, parent: "2", level: 1 },
    "21": { node: { id: "21", parent: "2" }, parent: "2", level: 1 },
    "22": { node: { id: "22", parent: "2" }, parent: "2", level: 1 },